# JennAI Project Boilerplate Notebook

This notebook provides a boilerplate setup for interacting with the `JennAI` monorepo project structure.
It demonstrates how to correctly configure paths, initialize logging, access configuration, utilize the dependency injection container, and integrate with project sub-layers (business, data, presentation), and interact with the Gemini API.

## 1. Environment Setup & Path Configuration

It's crucial that the notebook can correctly import modules from your `config/`, `core/`, and `project/` directories. This cell ensures the `JennAI` project root is added to `sys.path`.

In [ ]:
import sys
import os
from pathlib import Path

# Determine the JennAI project root dynamically
# Assumes this notebook is run from within the JennAI project or a subdirectory.
# It will navigate up until it finds a marker file like 'main.py' or 'environment.yaml'
# or if placed directly at root, it will be Path(__file__).resolve().parent

current_notebook_path = Path(__file__).resolve()
jennai_root = current_notebook_path
while not (jennai_root / 'main.py').exists() and not (jennai_root / 'environment.yaml').exists() and jennai_root.parent != jennai_root:
    jennai_root = jennai_root.parent

if str(jennai_root) not in sys.path:
    sys.path.insert(0, str(jennai_root)) # Insert at the beginning for higher priority

print(f"JennAI project root set to: {jennai_root}")
print(f"Current sys.path: {sys.path}")

## 2. Core Utilities: Logging & Configuration

Initialize the `loguru` logger and access global configuration settings.

In [ ]:
# Import the logging setup from config
from config.loguru_setup import setup_logging
from config.config import DEBUG_MODE # Assuming config.py has DEBUG_MODE

# Setup logging for the notebook session
# The log file will be created in JennAI/logs/jennai.log (relative to jennai_root)
setup_logging(debug_mode=DEBUG_MODE)
from loguru import logger

logger.info(f"Notebook Logging initialized. Running in DEBUG_MODE: {DEBUG_MODE}")
logger.debug("This is a debug message from the notebook.")
logger.warning("This is a warning message from the notebook.")

print(f"Debug mode from config: {DEBUG_MODE}")
print(f"Check your JennAI/logs/jennai.log file for these messages.")

## 3. Dependency Injection Setup

Instantiate and register simple dependencies using the `DependencyContainer`.

In [ ]:
from core.dependency_container import DependencyContainer

class MockService:
    def __init__(self, name="DefaultMock"):
        self.name = name
        logger.info(f"MockService '{self.name}' initialized.")

    def do_work(self):
        logger.info(f"MockService '{self.name}' doing some work.")
        return f"Work done by {self.name}"

global_container = DependencyContainer()
logger.info("Global DependencyContainer instantiated.")

# Register a dependency
global_container.register("mock_service", lambda: MockService("MyNotebookService"))
logger.info("Registered 'mock_service' with DependencyContainer.")

# Resolve the dependency
resolved_service = global_container.resolve("mock_service")
logger.info(f"Resolved 'mock_service': {resolved_service.name}")
print(resolved_service.do_work())

## 4. Gemini API Integration

Demonstrate importing and a placeholder for using the `gemini_api` module. Remember to set your `GOOGLE_API_KEY` environment variable.

In [ ]:
import os
from config.gemini_api import AIGenerator # Assuming AIGenerator is defined here

# Placeholder for your Google API Key
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    logger.warning("GOOGLE_API_KEY environment variable not set. Gemini API calls will fail.")
    # As a fallback for demonstration or if running outside proper env vars
    # You might temporarily set it here for testing, but remove for production!
    # google_api_key = "YOUR_ACTUAL_API_KEY_HERE"
else:
    logger.info("GOOGLE_API_KEY successfully loaded from environment.")


# Example: Register AIGenerator with the DependencyContainer
try:
    global_container.register(AIGenerator, lambda: AIGenerator(api_key=google_api_key))
    logger.info("AIGenerator registered with DependencyContainer.")
    
    # Resolve and use (example placeholder)
    gemini_instance = global_container.resolve(AIGenerator)
    logger.info("Resolved AIGenerator instance.")
    print("Gemini API (AIGenerator) configured and accessible through dependency injection.")
    # print(gemini_instance.generate_content("Hello, AI!")) # Uncomment to test actual API call
except Exception as e:
    logger.error(f"Failed to register/resolve AIGenerator: {e}")
    print(f"Gemini API setup failed: {e}")

## 5. Interacting with Project Sub-Layers (Business, Data, Presentation)

Demonstrate how to import and interact with modules from your `project/` subdirectories.

In [ ]:
# Example: Importing from project/business
# Assuming you have a dummy module like project/business/some_logic.py
try:
    # from project.business.some_logic import BusinessProcessor
    # processor = BusinessProcessor()
    # processor.process_data()
    logger.info("Successfully accessed project.business (placeholder).")
    print("Accessed project.business layer (conceptually).")
except ImportError:
    logger.warning("project.business.some_logic not found. This is a placeholder import.")
    print("project.business layer placeholder.")

# Example: Importing from project/data
# Assuming you have a dummy module like project/data/data_loader.py
try:
    # from project.data.data_loader import DataLoader
    # loader = DataLoader()
    # data = loader.load_source()
    logger.info("Successfully accessed project.data (placeholder).")
    print("Accessed project.data layer (conceptually).")
except ImportError:
    logger.warning("project.data.data_loader not found. This is a placeholder import.")
    print("project.data layer placeholder.")

# Example: Importing from project/presentation
# Assuming you have a dummy module like project/presentation/ui_handler.py
try:
    # from project.presentation.ui_handler import UIHandler
    # handler = UIHandler()
    # handler.render_view()
    logger.info("Successfully accessed project.presentation (placeholder).")
    print("Accessed project.presentation layer (conceptually).")
except ImportError:
    logger.warning("project.presentation.ui_handler not found. This is a placeholder import.")
    print("project.presentation layer placeholder.")

## 6. Interactive Testing / Verification

While `pytest` is for formal tests, notebooks are great for quick, interactive verification. Here, we can re-run parts of our `test_cuda.py` logic.

In [ ]:
import torch

print("--- Running Interactive CUDA Verification ---")
is_cuda_available = torch.cuda.is_available()
print(f"Is CUDA available: {is_cuda_available}")

if is_cuda_available:
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    logger.success("CUDA is successfully available from the notebook!")
    
    # Optional: Basic GPU tensor operation to confirm functionality
    try:
        tensor_on_gpu = torch.rand(3, 3, device='cuda')
        print(f"Successfully created tensor on GPU: {tensor_on_gpu.device}")
        logger.debug("GPU tensor creation successful.")
    except Exception as e:
        logger.error(f"Failed to create tensor on GPU: {e}")
        print(f"Failed to create tensor on GPU: {e}")
else:
    logger.warning("CUDA is not available. GPU-dependent operations will not work.")
    print("CUDA is NOT available.")

print("--- Interactive CUDA Verification Complete ---")

## 7. Cleanup / Final Notes

Remember to deactivate your Conda environment when you're done working in the terminal, or simply close Jupyter Lab.

This boilerplate should give you a strong starting point for interactive development within your JennAI monorepo!